In [1]:
import pandas as pd
import numpy as np
from google.colab import files

In [2]:
uploaded = files.upload()


Saving iris_dataset.csv to iris_dataset.csv


In [5]:
df = pd.read_csv('iris_dataset.csv')
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [18]:
feature_cols = ["sepal length (cm)", "sepal width (cm)", "petal length (cm)", "petal width (cm)"]

X = df[feature_cols]
y = df["target"]


In [47]:
grouped = df.groupby("target")

mean_tbl = grouped.mean()
std_tbl  = grouped.std()
rng_tbl  = grouped.max() - grouped.min()

print("mean:\n", mean_tbl)
print("\nstd:\n", std_tbl)
print("\nrng:\n", rng_tbl)


mean:
         sepal length (cm)  sepal width (cm)  petal length (cm)  \
target                                                           
0                   5.006             3.428              1.462   
1                   5.936             2.770              4.260   
2                   6.588             2.974              5.552   

        petal width (cm)  
target                    
0                  0.246  
1                  1.326  
2                  2.026  

std:
         sepal length (cm)  sepal width (cm)  petal length (cm)  \
target                                                           
0                0.352490          0.379064           0.173664   
1                0.516171          0.313798           0.469911   
2                0.635880          0.322497           0.551895   

        petal width (cm)  
target                    
0               0.105386  
1               0.197753  
2               0.274650  

rng:
         sepal length (cm)  sepal width (cm)  pe

In [48]:
grouped = df.groupby("target")["petal length (cm)"]

pl_mean = grouped.mean()
pl_std  = grouped.std()
pl_rng  = grouped.max() - grouped.min()

print("sepal length - mean:\n", pl_mean)
print("\nsepal length - std:\n", pl_std)
print("\nsepal length - rng:\n", pl_rng)

sepal length - mean:
 target
0    1.462
1    4.260
2    5.552
Name: petal length (cm), dtype: float64

sepal length - std:
 target
0    0.173664
1    0.469911
2    0.551895
Name: petal length (cm), dtype: float64

sepal length - rng:
 target
0    0.9
1    2.1
2    2.4
Name: petal length (cm), dtype: float64


In [49]:
from math import sqrt, pi, exp

def gaussian_pdf(x, mu, sigma):
    # sigma = std = sqrt(variance)
    return (1 / (sqrt(2*pi) * sigma)) * exp(-0.5 * ((x - mu) / sigma)**2)


In [51]:
classes = sorted(df["target"].unique())

def predict_one_petal_length(x):
    probs = []
    for c in classes:
        mu = pl_mean[c]
        sigma = pl_std[c]
        p = gaussian_pdf(x, mu, sigma)
        probs.append(p)
    return classes[int(np.argmax(probs))]


In [52]:
y_true = y.values
# y_true
y_train = [predict_one_petal_length(v) for v in df["petal length (cm)"].values]
# print(y_train)



In [53]:
correct = np.sum(np.array(y_train) == y_true)
precision_train_1feat = correct / len(y_true) * 100

precision_train_1feat

np.float64(95.33333333333334)

**Prediction on Sample Test**



In [54]:
uploaded_sample_test = files.upload()


Saving iris_test_samples.csv to iris_test_samples (1).csv


In [55]:
test_df = pd.read_csv('iris_test_samples.csv')
test_df

,sepal_length,sepal_width,petal_length,petal_width,label
0,5.1,3.4,1.5,0.2,0
1,4.9,2.9,1.4,0.1,0
2,5.4,3.9,1.7,0.4,0
3,6.1,2.8,4.0,1.3,1
4,5.9,3.0,4.2,1.5,1
5,6.4,3.1,4.7,1.4,1
6,6.9,3.2,5.5,2.0,2
7,6.5,3.0,5.8,2.2,2
8,7.1,3.0,5.9,2.1,2
9,6.8,2.8,5.1,1.9,2


In [56]:
test_pl = test_df["petal_length"].values
test_y_true = test_df["label"].values

test_y_pred_1feat = [predict_one_petal_length(v) for v in test_pl]

correct_test_1feat = np.sum(np.array(test_y_pred_1feat) == test_y_true)
precision_test_1feat = correct_test_1feat / len(test_y_true) * 100

precision_test_1feat


np.float64(100.0)

**Two Feature Test**

In [59]:
plw_mean = mean_tbl[["petal length (cm)", "petal width (cm)"]]
plw_std  = std_tbl[["petal length (cm)", "petal width (cm)"]]

print("plw_mean:\n", plw_mean)
print("\nplw_std:\n", plw_std)

plw_mean:
         petal length (cm)  petal width (cm)
target                                     
0                   1.462             0.246
1                   4.260             1.326
2                   5.552             2.026

plw_std:
         petal length (cm)  petal width (cm)
target                                     
0                0.173664          0.105386
1                0.469911          0.197753
2                0.551895          0.274650


In [68]:
def predict_two_features(pl, pw):
    probs = []
    for c in classes:
        mu_pl = plw_mean.loc[c, "petal length (cm)"]
        sd_pl = plw_std.loc[c, "petal length (cm)"]
        mu_pw = plw_mean.loc[c, "petal width (cm)"]
        sd_pw = plw_std.loc[c, "petal width (cm)"]

        p1 = gaussian_pdf(pl, mu_pl, sd_pl)
        p2 = gaussian_pdf(pw, mu_pw, sd_pw)
        probs.append(p1 * p2)
    return classes[int(np.argmax(probs))]


In [69]:
test_pl = test_df["petal_length"].values
test_pw = test_df["petal_width"].values
test_y_true = test_df["label"].values

test_y_pred_2feat = [predict_two_features(pl, pw)
                     for pl, pw in zip(test_pl, test_pw)]

correct_test_2feat = np.sum(np.array(test_y_pred_2feat) == test_y_true)
precision_test_2feat = correct_test_2feat / len(test_y_true) * 100

precision_test_2feat


np.float64(100.0)